In [9]:
import pandas as pd
from enum import Enum, auto
import datetime
import numpy as np

scheduler_file_loc = "../../data/scheduler_transition.csv"

## Questions:

1. What is 'begins' and 'ends' in this?

    answered: timestamps, just how I was checking if they were gt/lt didn't work

2. What is the relationship between 'begins' and 'time'?

In [2]:
class EventStates(Enum) :
    FORGOTTEN = 'forgotten'
    MEMORY = 'memory'
    PROCESSING = 'processing'
    QUEUED = 'queued'
    RELEASED = 'released'
    WAITING = 'waiting'

In [3]:
sched_x = pd.read_csv(scheduler_file_loc)


In [29]:
sched_x.columns

Index(['Unnamed: 0', 'key', 'thread', 'worker', 'prefix', 'group', 'start',
       'finish', 'stimulus_id', 'called_from', 'begins', 'ends', 'duration',
       'size', 'time'],
      dtype='object')

In [40]:
def pretty_print_line(entry) :
    for x in sched_x.columns :
        print("{column_name} : {column_value}".format(column_name = x, column_value = sched_x[x][entry]))

pretty_print_line(394)

Unnamed: 0 : 394
key : ('block-info-_map_read_frame-c50155175b03e4c1ec9664317841e345', 84, 0, 0, 0)
thread : nan
worker : nan
prefix : block-info
group : block-info-_map_read_frame-c50155175b03e4c1ec9664317841e345
start : waiting
finish : queued
stimulus_id : update-graph-1713455680.1067564
called_from : tcp://10.201.0.220:8786
begins : nan
ends : nan
duration : nan
size : nan
time : 1713455681.7593625


In [ ]:
class Event :
    def __init__() :
        return

In [19]:
def create_scheduler_event(sched_line) :
    start_state = EventStates(sched_line["start"])
    finish_state = EventStates(sched_line["finish"])

    time_datetime = datetime.datetime.fromtimestamp(sched_line["time"])
    if not np.isnan(sched_line['begins']) :
        start_datetime = datetime.datetime.fromtimestamp(sched_line['begins'])
    else :
        start_datetime = None
    if not np.isnan(sched_line['ends']) :
        end_datetime = datetime.datetime.fromtimestamp(sched_line['ends'])
    else : end_datetime = None

    print("Time: {time}\tStart: {start}\tEnds: {ends}".format(time=time_datetime, start=start_datetime, ends=end_datetime))
    print(time_datetime)

create_scheduler_event(sched_x.iloc(axis=0)[0])

Time: 2024-04-18 11:54:41.430816	Start: None	Ends: None
2024-04-18 11:54:41.430816


In [44]:
sched_x.shape

(46396, 15)

In [11]:
# Checking that beings is always g.t. ends, if not NaN.
nas = 0
negs = 0
pos = 0
for x in range(0, sched_x.shape[0]) :
    if np.isnan(sched_x['begins'][x]) :
        nas += 1
    else :
        if sched_x['ends'][x] - sched_x['begins'][x] < 0 :
            negs += 1
        else :
            pos += 1

print("Pos: {pos}\tNeg: {neg}\tNAs: {nas}".format(pos=pos, neg=negs, nas=nas))

Pos: 8911	Neg: 0	NAs: 37485


In [16]:
# Check whether begins always gt, eq of time
gts = 0
eqs = 0
lts = 0
nas = 0
for x in range(0, sched_x.shape[0]) :
    if not np.isnan(sched_x['begins'][x]) :
        if sched_x['begins'][x] > sched_x['time'][x] :
            gts += 1
        elif sched_x['begins'][x] == sched_x['time'][x] :
            eqs += 1
        else :
            lts += 1
    else :
        nas += 1

print("Gt: {gt}\tEq: {eq}\tLts: {lt}\tNa: {na}".format(gt=gts, eq=eqs, lt=lts, na=nas))

Gt: 653	Eq: 0	Lts: 8258	Na: 37485


In [45]:
sched_x['called_from'].value_counts()

called_from
tcp://10.201.0.220:8786    46396
Name: count, dtype: int64

In [11]:
states = set(sched_x['start'])
for state in sched_x['finish'] :
    states.add(state)
states

{'forgotten', 'memory', 'processing', 'queued', 'released', 'waiting'}